In [2]:
# Interactive Chart for distance from MRT by town
%matplotlib notebook
# testing rb codes for interactive
# https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Label
import ipywidgets as widgets
from IPython.display import display
import os
import bqplot.pyplot as bqp
import bqplot
import matplotlib.pyplot as plt
import numpy as np
import parseUnitDetails

# Import csv data file for HDB SBF Nov 2018
unitDetails = parseUnitDetails.getUnitDetails()


# Use this file to get town code
shortestMrt = np.genfromtxt("your_file.csv", delimiter=","
                ,skip_header=1,
                dtype=[('townCode','U5'),('mapCode','U5'),('ng','U24'),('blk','U6'),
                      ('blkX','f8'),('blkY','f8'),
                      ('closestMrtStation','U30'),('distToMrt','f8'),('town','U30')],
                   missing_values=['na','-',''],filling_values=0)

# labels of the 22 towns in SBF Nov 2018
labels = np.array(['Ang Mo Kio','Bedok','Bukit Batok','Bukit Merah','Bukit Panjang','Choa Chu Kang',
                'Clementi','Geylang','Hougang','Jurong East','Jurong West','Kallang / Whampoa','Pasir Ris','Punggol',
                'Queenstown','Sembawang','Sengkang','Serangoon','Tampines','Toa Payoh', 'Woodlands','Yishun'])




## Dummy data
townNames = ['Ang Mo Kio','Bedok','Bukit Batok','Bukit Merah','Bukit Panjang','Choa Chu Kang',
                'Clementi','Geylang','Hougang','Jurong East','Jurong West','Kallang / Whampoa','Pasir Ris','Punggol',
                'Queenstown','Sembawang','Sengkang','Serangoon','Tampines','Toa Payoh', 'Woodlands','Yishun']

# include the tp2,tp3 (merged data merged tp2, tp3 into just tp. so in order to call out correct townmap, we need to define townNamesIncTp23)
townNamesIncTp23 = ['Ang Mo Kio','Bedok','Bukit Batok','Bukit Merah','Bukit Panjang','Choa Chu Kang',
                'Clementi','Geylang','Hougang','Jurong East','Jurong West','Kallang / Whampoa','Pasir Ris','Punggol',
                'Queenstown','Sembawang','Sengkang','Serangoon','Tampines',
                'Toa Payoh', 'Toa Payoh (Upper Aljunied)', 'Toa Payoh (Bidadari)', 'Woodlands','Yishun']

allMapCodes = ['amk', 'bd', 'bb', 'bm', 'bp', 'cck', 'cl', 'gl', 'hg', 'je', 'jw', 
           'kwn', 'prc', 'pg', 'qt', 'sb', 'sk', 'sgn', 'tap', 'tp', 'tp2', 'tp3', 'wl', 'ys']


dict_TownDisplayName_toMapCode = dict(zip(townNamesIncTp23, allMapCodes))
dict_mapCode_to_townDisplayName = dict(zip(allMapCodes, townNamesIncTp23))

#print(dict_mapCode_to_townDisplayName)

# Back to first data type
DistToMrt =[]  
blkNum =[] 
mapCodes = [] 

for unit in unitDetails:
    thisTown = unit['town'] # single value
    thisBlk = unit['blk']
    
    
    # get mapCode from another dataset
    # by equating both town and blk to get the mapCode
    
    #print('equating town:', thisTown, ', blk: ', thisBlk)
    
    subsetSameTown = shortestMrt[shortestMrt['town'] == thisTown] #shortestMrt[[True, False,...]]
    #print('subsetSameTown ', subsetSameTown)
    
    subsetSameTownBlk = subsetSameTown[subsetSameTown['blk'] == thisBlk]
  
    thisMapCode = subsetSameTownBlk[0]['mapCode'].strip() # get the 1st row, 
                                                  # may have repeated rows, they are the same anyway
                                                  # strip to get rid of extra space/linebreak just in case
    #print(thisMapCode)
    
    DistToMrt.append(unit['DistToMrt'])
    blkNum.append(thisBlk)
    mapCodes.append(thisMapCode)
    


blkDetails = list(zip(mapCodes, blkNum, DistToMrt))
#print(blkDetails)



# Radio buttons for towns
wRadioTowns = widgets.RadioButtons(
#    options=['pepperoni', 'orange', 'anchovies'],
#     value='pineapple',
    description='HDB Towns',
    disabled=False
) 

# given blk data and distance
def f(x=900):
    #listOfBlkWithDistLessThan = [blkNum for townCode,blkNum,distToMrt in blkDetails if distToMrt <= x]
    tmp = [townCode for townCode,blkNum,distToMrt in blkDetails if distToMrt <= x] #find the "town"
    listOfTownCodeWithDistLessThan = list(set(tmp)) # get unique townCodes using tmp
    #print(x)
    #print(w.value)
    fullTownNameList = [dict_mapCode_to_townDisplayName[x] for x in listOfTownCodeWithDistLessThan]
    fullTownNameList.sort() # sort by alphabet order
    
    wRadioTowns.options = fullTownNameList # this is for the wRadioTowns options arg, lib style
    #return listOfBlkWithDistLessThan # will print out
    return None  # None is the NULL in python


# Set up the slider format
minDist = min(DistToMrt)
maxDist = max(DistToMrt)
stepDist = 1
defaultValue = 900
#print('minDist:', minDist, ' maxDist: ', maxDist)
distSlider = widgets.IntSlider(min=minDist,max=maxDist,step=stepDist, value=defaultValue)
slider = interact(f, x=distSlider) # func f is passed inside, 3rd arg gets passed to f

# show the map
def showTownMap(townFriendlyName):
    #print('showTownMap: %s' % townCode)
    if not(townFriendlyName is None):
        townCode = dict_TownDisplayName_toMapCode[townFriendlyName]
        image_path = os.path.abspath('./townmap/townmap/townmap_%s.png' % (townCode,))  #"./" is current folder
        # s is for the townCode string
        #print(image_path)

        bqp.close(1) #close old figure
        # https://stackoverflow.com/questions/37144008/change-size-of-fig-in-bqplot
        fig = bqp.figure(key=1) #open figure 1
        fig.layout.width = '1000px'
        fig.layout.height = '1000px'
        bqp.imshow(image_path, 'filename') #open by filename
        bqp.show()


# your new function for click
def wRadioTowns_onChange(a):  # a is a variable name
    townNameSelected = a['owner'].value # value is attribute of 'owner'
    #print('townNameSelected', townNameSelected)
    if townNameSelected is not None:
        townCodeSelected = dict_TownDisplayName_toMapCode[townNameSelected]
    else:
        townCodeSelected = None
    distToMrt = distSlider.value
    #print('wRadioTowns_onChange: townCode: %s, distToMrt: %s' % (townCode, distToMrt))
    #print('a', a)
    listOfBlkSubset = [blkNum for townCode,blkNum,distToMrt in blkDetails if (distToMrt <= distToMrt) and (townCode == townCodeSelected)]
    #listOfBlkSubset = [blkNum for townCode, blkNum in listOfBlkSubsetTmp if townCode == townCodeSelected]
    #print('listOfBlkSubsetTmp', townCodeSelected, listOfBlkSubsetTmp)
    listOfBlkSubsetUnique = list(set(listOfBlkSubset))
    listOfBlkSubsetUnique.sort() # sort
   
    
    # if using widgets.Select
    #listOfBlkSubsetUnique.append('') # need a selected value, so add dummy, pad 6 space for the init width
    wBlkDisplay.options = listOfBlkSubsetUnique
    
    #print(listOfBlkSubset)
    showTownMap(townNameSelected)  #the differnce from another code is this
    #pass  # 'continue' means continue in loop. But 'pass'

wRadioTowns.observe(wRadioTowns_onChange)
display(wRadioTowns)



#style = {'description_width': 'initial'}
wBlkDisplay = widgets.Select(
    options=[''],
    value=None,
    rows=10,
    #description='List of blks meeting distance and town criteria',
    description='',
    disabled=True,
    #style=style
)
#display(wBlkDisplay)
HBox([Label('List of blks meeting distance and town criteria'), wBlkDisplay])

interactive(children=(IntSlider(value=900, description='x', max=2489, min=44), Output()), _dom_classes=('widge…

RadioButtons(description='HDB Towns', options=('Ang Mo Kio', 'Bedok', 'Bukit Batok', 'Bukit Merah', 'Bukit Pan…